In [ ]:
from google.colab import files
uploaded = files.upload()


Saving True.csv (1).zip to True.csv (1).zip
Saving Fake.csv (1).zip to Fake.csv (1).zip


In [ ]:
import zipfile

# Unzip Fake.csv
with zipfile.ZipFile("Fake.csv (1).zip", 'r') as zip_ref:
    zip_ref.extractall()

# Unzip True.csv
with zipfile.ZipFile("True.csv (1).zip", 'r') as zip_ref:
    zip_ref.extractall()


In [ ]:
import pandas as pd

# Load datasets
fake_df = pd.read_csv("Fake.csv")
true_df = pd.read_csv("True.csv")

# Add labels
fake_df['label'] = 'FAKE'
true_df['label'] = 'REAL'

# Combine and shuffle
combined_df = pd.concat([fake_df, true_df])
combined_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Display
combined_df.head()


,title,text,subject,date,label
0,Ben Stein Calls Out 9th Circuit Court: Committ...,"21st Century Wire says Ben Stein, reputable pr...",US_News,"February 13, 2017",FAKE
1,Trump drops Steve Bannon from National Securit...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"April 5, 2017",REAL
2,Puerto Rico expects U.S. to lift Jones Act shi...,(Reuters) - Puerto Rico Governor Ricardo Rosse...,politicsNews,"September 27, 2017",REAL
3,OOPS: Trump Just Accidentally Confirmed He Le...,"On Monday, Donald Trump once again embarrassed...",News,"May 22, 2017",FAKE
4,Donald Trump heads for Scotland to reopen a go...,"GLASGOW, Scotland (Reuters) - Most U.S. presid...",politicsNews,"June 24, 2016",REAL


In [ ]:
import string
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))  # remove punctuation
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return " ".join(words)

# Clean the text column
combined_df['text'] = combined_df['title'] + " " + combined_df['text']  # merge title + text
combined_df['text'] = combined_df['text'].astype(str).apply(clean_text)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_df=0.7, stop_words='english')
X = vectorizer.fit_transform(combined_df['text'])
y = combined_df['label']


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.983630289532294

Classification Report:
               precision    recall  f1-score   support

        FAKE       0.99      0.98      0.98      4710
        REAL       0.98      0.99      0.98      4270

    accuracy                           0.98      8980
   macro avg       0.98      0.98      0.98      8980
weighted avg       0.98      0.98      0.98      8980



In [ ]:
import pickle

# Save the trained model
with open('fake_news_model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

# Save the TF-IDF vectorizer
with open('tfidf_vectorizer.pkl', 'wb') as vec_file:
    pickle.dump(vectorizer, vec_file)

# Download the files to your PC
from google.colab import files
files.download('fake_news_model.pkl')
files.download('tfidf_vectorizer.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import streamlit as st
import pickle

# Load the saved model and vectorizer
with open('fake_news_model.pkl', 'rb') as model_file:
    model = pickle.load(model_file)

with open('tfidf_vectorizer.pkl', 'rb') as vec_file:
    vectorizer = pickle.load(vec_file)

# Streamlit interface
st.title("📰 Fake News Detection App")

st.markdown("Enter a news article below and let the app determine if it's **REAL** or **FAKE**.")

news_text = st.text_area("Enter the news article content here:")

if st.button("Predict"):
    if news_text.strip() == "":
        st.warning("Please enter some text to analyze.")
    else:
        transformed = vectorizer.transform([news_text])
        prediction = model.predict(transformed)
        prediction_proba = model.predict_proba(transformed)

        confidence = max(prediction_proba[0]) * 100

        if prediction[0] == "REAL":
            st.success(f"✅ This news is REAL with {confidence:.2f}% confidence.")
            st.progress(confidence / 100)
        else:
            st.error(f"❌ This news is FAKE with {confidence:.2f}% confidence.")
            st.progress(confidence / 100)



FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\poulo\\OneDrive\\Desktop\\FakeNewsApp\\fake_news_model.pkl'